- [Summary of the MIMIC-III Intensice Care Unit Data](https://baeseongsu.github.io/posts/mimiciii/)
- [Exploring the Basic Tables of MIMIC-III](https://mimic.mit.edu/docs/iii/tables/)
- [Checking the MIMIC-III Schema](https://mit-lcp.github.io/mimic-schema-spy/)
- [Website for Searching ICD9_CODE](http://www.icd9data.com/2012/Volume1/460-519/480-488/482/482.41.htm)

In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore')

# Pneumonia Case Extraction

In [2]:
# patient = pd.read_csv('/data/MIMIC_III/PATIENTS.csv')
lab = pd.read_csv('/data/MIMIC_III/LABEVENTS.csv')
icd = pd.read_csv('/data/MIMIC_III/D_ICD_DIAGNOSES.csv') # disease information
patient_icd = pd.read_csv('/data/MIMIC_III/DIAGNOSES_ICD.csv') # Patient-specific disease information 

In [3]:
# 1. Extract diseases related to pneumonia -> 95 types
pneu_list = icd[(icd['SHORT_TITLE'].str.contains('pneum'))|icd['SHORT_TITLE'].str.contains('Pneum')]['ICD9_CODE']

# 2. Extract patients with pneumonia-related diseases -> 14,159 patients
all_pneu_patient = patient_icd[patient_icd['ICD9_CODE'].isin(pneu_list)]

# 3. Extract patients corresponding to the top 3 pneumonia types
top3_pneu = all_pneu_patient['ICD9_CODE'].value_counts()[:3].index
top3_pneu_patient = all_pneu_patient[all_pneu_patient['ICD9_CODE'].isin(top3_pneu)]

# Confirm the exact names of diseases for ICD9_CODE: 486, 5070, 48241
# -> 486 : Pneumonia, organism unspecified
# -> 5070 : Pneumonia due to food or vomit
# -> 48241 : Methicillin susceptible pneumonia due to Staphylococcus aureus
icd[icd['ICD9_CODE'].isin(top3_pneu)]

# 4. Patients in top3_pneu_patient may have multiple hospital records 
# -> Unique patient count: 7807
patient_list = top3_pneu_patient['SUBJECT_ID'].unique()

# 5. Extract information from labevents for patients in top3_pneu_patient
patient_lab = lab[lab['SUBJECT_ID'].isin(patient_list)].reset_index(drop=True)

# 6. Fill missing values in the 'FLAG' column with 'nan', then check distribution
# Without setting to 'nan', value_counts() won't count the number of NaNs
patient_lab['FLAG'] = patient_lab['FLAG'].fillna('nan')
patient_lab['FLAG'].value_counts()

# 7. The number of pneumonia patients in patient data is 8 more than in labevent data
# Outpatients do not have 'HADM_ID' in labevent 
patient_lab_list = patient_lab['SUBJECT_ID'].unique()
outpatient_idx = list(set(patient_list) - set(patient_lab['SUBJECT_ID'].unique()))
outpatient_idx

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
5129,5509,48241,Meth sus pneum d/t Staph,Methicillin susceptible pneumonia due to Staph...
5147,5528,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified"
5407,5136,5070,Food/vomit pneumonitis,Pneumonitis due to inhalation of food or vomitus


nan         5931403
abnormal    3406609
delta         18337
Name: FLAG, dtype: int64

[60961, 48968, 17674, 9388, 19097, 93114, 58012, 95230]

# Extract ITEMID from LAB, PRE, PRO to create X data
- `X` : (7727, 10, 4068) = (PATIENTS, TIMEPOINTS, FEATURES)


- LAB : `SUBJECT_ID`, `CHARTTIME`, `ITEMID`, `FLAG`
- PRO : `SUBJECT_ID`, `STARTTIME`, `ENDTIME`, `ITEMID`
- PRE : `SUBJECT_ID`, `STARTDATE`, `ENDDATE`, `NDC`


- Unlike LAB, PRO and PRE require additional preprocessing based on start and end dates
- Final columns = [`SUBJECT_ID`, `ITEMID`, `CHARTTIME`, `TYPE`]

In [ ]:
# Function to keep only the date
import datetime as dt
def date_only(df, x):
    df[x] = pd.to_datetime(df[x])
    df[x] = df[x].dt.date

## LAB

In [5]:
lab = pd.read_csv('/data/MIMIC_III/LABEVENTS.csv')

In [6]:
lab2 = lab[lab['SUBJECT_ID'].isin(patient_lab_list)]
lab2 = lab2[['SUBJECT_ID','CHARTTIME', 'ITEMID', 'FLAG']]

# Extract only the records where FLAG is 'abnormal'
lab2 = lab2[lab2['FLAG']=='abnormal']
lab2 = lab2.drop(columns = ['FLAG'])
date_only(lab2, 'CHARTTIME')
lab2 = lab2.sort_values(by = 'SUBJECT_ID')
lab2['TYPE'] = 'LAB'

lab2 = lab2.reset_index(drop=True)

In [7]:
lab2 = lab2.drop_duplicates()
lab2

,SUBJECT_ID,CHARTTIME,ITEMID,TYPE
0,9,2149-11-14,50821,LAB
1,9,2149-11-13,50910,LAB
2,9,2149-11-13,50893,LAB
3,9,2149-11-13,50882,LAB
4,9,2149-11-13,50818,LAB
...,...,...,...,...
3406603,99985,2181-02-05,50931,LAB
3406604,99985,2181-02-05,51006,LAB
3406605,99985,2181-02-05,51221,LAB
3406607,99985,2181-02-01,51244,LAB


## PRE

In [8]:
pre = pd.read_csv('/data/MIMIC_III/PRESCRIPTIONS.csv')

In [9]:
pre2 = pre[pre['SUBJECT_ID'].isin(patient_lab_list)]
pre2 = pre2[['SUBJECT_ID', 'STARTDATE', 'ENDDATE', 'NDC']]
date_only(pre2, 'STARTDATE')
date_only(pre2, 'ENDDATE')

# Remove rows where both STARTDATE and ENDDATE are null
both_null = pre2[(pre2['STARTDATE'].isnull()) & (pre2['ENDDATE'].isnull())].index
pre2 = pre2.drop(index=both_null)

# Remove rows where NDC is null
ndc_null = pre2[(pre2['NDC'].isnull())].index
pre2 = pre2.drop(index=ndc_null)

# If ENDDATE is null, replace it with STARTDATE
end_null = pre2[pre2['ENDDATE'].isnull()]
end_null['ENDDATE'] = end_null['STARTDATE']
pre2.loc[end_null.index] = end_null

# If STARTDATE is null, replace it with ENDDATE
start_null = pre2[pre2['STARTDATE'].isnull()]
start_null['STARTDATE'] = start_null['ENDDATE']
pre2.loc[start_null.index] = start_null

pre2 = pre2.sort_values(by=['SUBJECT_ID', 'STARTDATE'])
pre2 = pre2.reset_index(drop=True)

pre2

,SUBJECT_ID,STARTDATE,ENDDATE,NDC
0,9,2149-11-09,2149-11-09,0.0
1,9,2149-11-09,2149-11-09,85036207.0
2,9,2149-11-09,2149-11-09,456066270.0
3,9,2149-11-09,2149-11-09,456066270.0
4,9,2149-11-09,2149-11-09,338001702.0
...,...,...,...,...
1327244,99985,2181-02-09,2181-02-12,0.0
1327245,99985,2181-02-09,2181-02-12,338101948.0
1327246,99985,2181-02-09,2181-02-12,8084199.0
1327247,99985,2181-02-09,2181-02-12,0.0


In [10]:
from datetime import datetime, timedelta
from tqdm import tqdm

def date_range(start, end):
    dates = [(start + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end-start).days+1)]
    return dates

lst_time = []
lst_itemid = []
lst_subid = []

pre2_list = pre2.values.tolist()
for idx, row in enumerate(tqdm(pre2_list)):
    sub, start, end, itemid = row[0], row[1], row[2], row[3]
    
    # CHARTTIME
    day_list = date_range(start, end)    
    lst_time.extend(day_list)
    
    # ITEMID 
    lst_itemid.extend([itemid] * len(day_list))
    
    # SUBJECT_ID
    lst_subid.extend([sub] * len(day_list))

100%|██████████| 1327249/1327249 [00:13<00:00, 98633.78it/s] 


In [11]:
pre2sub = pd.DataFrame(lst_subid)
pre2time = pd.DataFrame(lst_time)
pre2item = pd.DataFrame(lst_itemid)

pre2 = pd.concat([pre2sub, pre2time, pre2item], axis = 1)
pre2.columns = ['SUBJECT_ID', 'CHARTTIME', 'ITEMID']
pre2 = pre2.drop_duplicates()
pre2['TYPE'] = 'PRE'

pre2

,SUBJECT_ID,CHARTTIME,ITEMID,TYPE
0,9,2149-11-09,0.0,PRE
1,9,2149-11-09,85036207.0,PRE
2,9,2149-11-09,456066270.0,PRE
4,9,2149-11-09,338001702.0,PRE
5,9,2149-11-09,74302401.0,PRE
...,...,...,...,...
5650360,99985,2181-02-10,8084199.0,PRE
5650361,99985,2181-02-11,8084199.0,PRE
5650362,99985,2181-02-12,8084199.0,PRE
5650367,99985,2181-02-11,781305714.0,PRE


## PRO

In [12]:
pro = pd.read_csv('/data/MIMIC_III/PROCEDUREEVENTS_MV.csv')

In [13]:
pro2 = pro[pro['SUBJECT_ID'].isin(patient_lab_list)]
pro2 = pro2[['SUBJECT_ID', 'STARTTIME', 'ENDTIME', 'ITEMID']]
date_only(pro2, 'STARTTIME')
date_only(pro2, 'ENDTIME')

# Remove rows where both STARTTIME and ENDTIME are null
both_null = pro2[(pro2['STARTTIME'].isnull())&(pro2['ENDTIME'].isnull())].index
pro2 = pro2.drop(index=both_null)

# Remove rows where ITEMID is null
ndc_null = pro2[(pro2['ITEMID'].isnull())].index
pro2 = pro2.drop(index=ndc_null)

# If ENDTIME is null, replace it with STARTTIME
end_null = pro2[pro2['ENDTIME'].isnull()]
end_null['ENDTIME'] = end_null['STARTTIME']
pro2.loc[end_null.index] = end_null

# If STARTTIME is null, replace it with ENDTIME
start_null = pro2[pro2['STARTTIME'].isnull()]
start_null['STARTTIME'] = start_null['ENDTIME']
pro2.loc[start_null.index] = start_null

pro2 = pro2.sort_values(by = ['SUBJECT_ID','STARTTIME'])
pro2 = pro2.reset_index(drop = True)

In [14]:
pro2 = pro2.drop_duplicates()
pro2

,SUBJECT_ID,STARTTIME,ENDTIME,ITEMID
0,36,2134-05-12,2134-05-15,224275
1,36,2134-05-12,2134-05-12,225402
2,36,2134-05-12,2134-05-12,221214
4,36,2134-05-12,2134-05-12,225432
5,36,2134-05-12,2134-05-12,224385
...,...,...,...,...
79672,99985,2181-02-03,2181-02-03,225454
79673,99985,2181-02-04,2181-02-04,225814
79674,99985,2181-02-05,2181-02-05,225459
79675,99985,2181-02-07,2181-02-07,227194


In [15]:
from datetime import datetime, timedelta
from tqdm import tqdm

def date_range(start, end):
    dates = [(start + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end-start).days+1)]
    return dates

lst_time = []
lst_itemid = []
lst_subid = []

pro2_list = pro2.values.tolist()
for idx, row in enumerate(tqdm(pro2_list)):
    sub, start, end, itemid = row[0], row[1], row[2], row[3]
    
    # CHARTTIME
    day_list = date_range(start, end)    
    lst_time.extend(day_list)
    
    # ITEMID 
    lst_itemid.extend([itemid] * len(day_list))
    
    # SUBJECT_ID
    lst_subid.extend([sub] * len(day_list))

100%|██████████| 71052/71052 [00:00<00:00, 162183.35it/s]


In [16]:
pro2sub = pd.DataFrame(lst_subid)
pro2time = pd.DataFrame(lst_time)
pro2item = pd.DataFrame(lst_itemid)

pro2 = pd.concat([pro2sub, pro2time, pro2item], axis = 1)
pro2.columns = ['SUBJECT_ID', 'CHARTTIME', 'ITEMID']
pro2 = pro2.drop_duplicates()
pro2['TYPE'] = 'PRO'

pro2

,SUBJECT_ID,CHARTTIME,ITEMID,TYPE
0,36,2134-05-12,224275,PRO
1,36,2134-05-13,224275,PRO
2,36,2134-05-14,224275,PRO
3,36,2134-05-15,224275,PRO
4,36,2134-05-12,225402,PRO
...,...,...,...,...
152377,99985,2181-02-04,225814,PRO
152378,99985,2181-02-05,225459,PRO
152379,99985,2181-02-07,227194,PRO
152380,99985,2181-02-08,224264,PRO


# `X` : (7799, 10, 4069)

In [17]:
m1 = pd.merge(lab2, pre2, how = 'outer')
total_data = pd.merge(m1, pro2, how = 'outer')
total_data = total_data.sort_values(['SUBJECT_ID','CHARTTIME']).reset_index(drop=True)
total_data = total_data.astype({'ITEMID':'int'})
total_data

# total_data.to_csv('total_data.csv', index = False)

,SUBJECT_ID,CHARTTIME,ITEMID,TYPE
0,9,2149-11-09,50822,LAB
1,9,2149-11-09,50821,LAB
2,9,2149-11-09,50813,LAB
3,9,2149-11-09,50809,LAB
4,9,2149-11-09,50808,LAB
...,...,...,...,...
6895977,99985,2181-03-06,51256,LAB
6895978,99985,2182-03-14,51279,LAB
6895979,99985,2182-03-14,51222,LAB
6895980,99985,2182-03-14,51221,LAB


In [3]:
total_data = pd.read_csv('total_data.csv')

# Function to keep only the date
import datetime as dt
def date_only(df, x):
    df[x] = pd.to_datetime(df[x])
    df[x] = df[x].dt.date

dic_sub2idx = {}
for i, j in enumerate(total_data['SUBJECT_ID'].unique()):
    dic_sub2idx[j] = i

dic_item2idx = {}
for i, j in enumerate(total_data['ITEMID'].sort_values().unique()):
    dic_item2idx[j] = i

admission = pd.read_csv('/data/MIMIC_III/ADMISSIONS.csv')
admission = admission[admission['SUBJECT_ID'].isin(total_data['SUBJECT_ID'].unique())]

# Remove time from DISCHTIME column
date_only(admission, 'DISCHTIME')
dic_sub2final_date = dict(admission.groupby('SUBJECT_ID')['DISCHTIME'].max())

In [4]:
from datetime import timedelta
from tqdm import tqdm
import datetime

x = np.zeros((7799,10,4069))

IF = open('total_data.csv','r')
line = IF.readline()
for line in tqdm(IF):
    ss = line.strip('\n').split(',')
    sub, item, charttime = int(ss[0]), int(ss[2]), datetime.date.fromisoformat(ss[1])
    subidx = dic_sub2idx[sub]
    itemidx = dic_item2idx[item]
    finaldate = dic_sub2final_date[sub]
    dateidx = -(finaldate - charttime).days + 10
    
    if (dateidx < 0) or (dateidx > 9):
        continue
    else:
        x[subidx, dateidx, itemidx] = 1
        
IF.close()
print(x.sum())

6895982it [00:11, 585769.68it/s]


2042722.0


# `X` : (7727, 10, 4068)
- When constructing 3D data based on total_data, some SUBJECT_IDs may be excluded because only the days from D-1 to D-10 relative to the discharge date are considered.
- There are a total of 72 sub_ids that have a value of 0 for all timepoints and item_ids.
- By excluding these 72 sub_ids and removing item_ids that do not apply to any of the remaining individuals, `X` becomes (7727, 10, 4068).


- Originally, both steps 1 and 2 should be performed, but there was almost no difference in LSTM model predictive performance when step 2 was additionally executed -> maintaining (7727, 10, 4068) is ok
    - Refer to the test with x_(7727, 10, 3595) below

In [5]:
# 1. Removing data that has a value of 0 for all entries for each sub_id (From 7799 to 7727 individuals)
sub_sum = x.sum(axis=1).sum(axis=1)
sub_sum[sub_sum > 0] = 1
sub_sum = pd.DataFrame(sub_sum)
zero_index = sub_sum[sub_sum[0]==0].index
new_x = np.delete(x, zero_index, axis = 0)
new_x.shape
new_x.sum()

(7727, 10, 4069)

2042722.0

In [6]:
# 2. Removing data that has a value of 0 for all entries for each feature (From 4069 to 3595 features)
f_sum = new_x.sum(axis=0).sum(axis=0) 
f_sum[f_sum > 0] = 1
f_sum = pd.DataFrame(f_sum)
zero_index2 = f_sum[f_sum[0]==0].index
final_x = np.delete(new_x, zero_index2, axis = 2)
final_x.shape
final_x.sum()

(7727, 10, 3595)

2042722.0

In [38]:
# x save
# np.save('x_(7727,10,3595).npy', final_x)
# np.save('x_(7727,10,4069).npy', new_x)

# `x` : (7727, 10, 3595) test

- x_(7727, 10, 3595) - Single LSTM
    - `accuracy` : 0.754, `precision` : 0.787, `recall` : 0.810, `f1` : 0.798, `roc_auc` : 0.740


- x_(7727, 10, 4068) - Single LSTM
    - `accuracy` : 0.755, `precision` : 0.799, `recall` : 0.792, `f1` : 0.795, `roc_auc` : 0.746
    
    
- x_(7799, 10, 4069) - Single LSTM
    - `accuracy` : 0.747, `precision` : 0.799, `recall` : 0.796, `f1` : 0.798, `roc_auc` : 0.730

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings 
warnings.filterwarnings(action='ignore')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        print(e)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
import random  
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7799,10,4069).npy')
y = np.load('/project/LSH/y_(7799,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6239, 10, 4069), (6239,), (1560, 10, 4069), (1560,))

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
from tensorflow import keras
import random  
from tensorflow.keras.callbacks import EarlyStopping

# LSTM
def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.01), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return lstm

with tf.device('/device:GPU:0'):
    print("Single LSTM Start")
#     tf.random.set_seed(0)
    model = get_model()
    
    early_stop = EarlyStopping(monitor='val_loss', patience=30, verbose=1, restore_best_weights=False)
    model.fit(X_train, y_train, epochs=300, batch_size=1024, validation_split=0.25, callbacks=[early_stop])
    preds = model.predict(X_test)

    preds[preds>0.5]=1
    preds[preds<=0.5]=0
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f'accuracy : {acc:.3f}, precision : {precision:.3f}, recall : {recall:.3f}, f1 : {f1:.3f}, roc_auc : {roc_auc:.3f}')

Single LSTM Start
Epoch 1/300
5/5 [==============================] - 5s 370ms/step - loss: 0.6933 - acc: 0.5734 - val_loss: 0.6763 - val_acc: 0.5917
Epoch 2/300
5/5 [==============================] - 1s 266ms/step - loss: 0.6768 - acc: 0.5933 - val_loss: 0.6770 - val_acc: 0.5917
Epoch 3/300
5/5 [==============================] - 1s 270ms/step - loss: 0.6732 - acc: 0.6065 - val_loss: 0.6804 - val_acc: 0.5917
Epoch 4/300
5/5 [==============================] - 1s 271ms/step - loss: 0.6705 - acc: 0.6155 - val_loss: 0.6800 - val_acc: 0.5917
Epoch 5/300
5/5 [==============================] - 1s 274ms/step - loss: 0.6677 - acc: 0.6153 - val_loss: 0.6754 - val_acc: 0.5917
Epoch 6/300
5/5 [==============================] - 1s 270ms/step - loss: 0.6669 - acc: 0.6121 - val_loss: 0.6729 - val_acc: 0.5917
Epoch 7/300
5/5 [==============================] - 1s 270ms/step - loss: 0.6658 - acc: 0.6121 - val_loss: 0.6676 - val_acc: 0.5917
Epoch 8/300
5/5 [==============================] - 1s 260ms/step 

accuracy : 0.747, precision : 0.799, recall : 0.796, f1 : 0.798, roc_auc : 0.730
